# Python for Spatial Analysis - Spatial Clustering
### Second part of the module of GG3209 Spatial Analysis with GIS.
---
Dr Fernando Benitez -  University of St Andrews - School of Geography and Sustainable Development

# Introduction

We reach to the final lecture of this short introducction to python. To finish I would like to present two methods that have been relevant to the spatial analysys field.

## K-means and DBSCAN in Spatial Data Analysis

One of the main reasons for us to lead this module is to describe you why the Spatial data analysis plays an important role in understanding patterns, trends, and relationships within geographical datasets. As we mentioned in the lecture there are two popular clustering algorithms, K-means and DBSCAN, are widely used in spatial data analysis for uncovering meaningful insights. Let's explore what each algorithm describe and discuss their relevance in the field.

### K-means Clustering:

It is a *partitioning* method that aims to group data points into 'k' clusters based on similarity (did you recall the Tobler's law?). It operates by iteratively assigning data points to clusters and updating cluster centroids until convergence. K-means is particularly useful for spatial data analysis as it helps identify spatial patterns and groupings within a dataset. It is widely applied in fields such as urban planning, crime analysis, and environmental science.

In the context of spatial analysis, K-means can assist in identifying areas of similar characteristics, such as concentrations of certain activities or the segmentation of regions based on specific features. The method can be also be applied to multidimensional datasets and is used to applied to an introduction to spatial clustering. There are pros and cons that comes with this method, that will be covered in the advance module.

### DBSCAN (Density-Based Spatial Clustering of Applications with Noise)

It is a *density-based* clustering algorithm that identifies clusters based on the density of data points in space. Unlike K-means, DBSCAN can discover clusters of arbitrary shapes and sizes. It is especially valuable in spatial data analysis because it can detect clusters in regions with varying data point densities.

In the spatial analysis field, DBSCAN is often used to find spatial hotspots, anomalies, or areas with varying population densities. It can be effective in identifying spatial patterns that may not conform to standard geometrical shapes.

It is a good practice to try out both methods and start to compare both clustering to define where are the best results.

## Why these methods are relevant in the Spatial Data Analysis field.

Both K-means and DBSCAN are indispensable tools in the spatial data analyst's toolkit, lets enumerate three initial use cases:

- **Pattern Recognition:** These algorithms help recognize spatial patterns, which can be vital for urban planning, environmental monitoring, and resource allocation.

- **Anomaly Detection:** They can identify outliers and anomalies in spatial datasets, enabling the detection of irregularities or unexpected phenomena.

- **Segmentation:** Clustering aids in segmenting spatial regions based on certain characteristics, facilitating targeted analysis and decision-making.

In this notebook, we will implement **K-means** and **DBSCAN** on synthetic and real-world datasets, providing hands-on experience with spatial clustering techniques.

# Mount your Drive

As we need to access our Google Drive unit, to get access to the data we previously upload., we need to mount that drive as part of this current python session. Run the following code to allow this notebook access to your drive.

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Installing aditional libraries

There are some new libraries we need to install to before we jump into the examples. [Lonboard](https://developmentseed.org/lonboard/latest/) will help us to represent big datasets (handles more than a millon of records). As this is a practice we will work with a partially small dataset from the reported crimes in London.

In [ ]:
pip install lonboard

# Importing the libraries

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import geopandas as gpd
# This is a super popular library for working with ML and DL algorithms.
from sklearn.cluster import KMeans, DBSCAN
import matplotlib.pyplot as plt
import shapely
import folium
import seaborn as sns
from lonboard import Map, ScatterplotLayer, viz

# K-Means - manual implementation

Let's start with a basic example using synthetic data (like fake data we can juts use for testing) and just python specifically numpy, to run the K-Means algoritmn from scratch, step by step. We will create a several functions (now you are familiar with functions in python and numpy). The proces is the following:



**Function No 1** - `initialize_centroids`: This function initializes the centroids for the K-means algorithm.

**Input:**
- data: The input data points.
- k: The number of clusters.

**Implementation:**
Randomly selects `k `indices from the data without replacement.
Returns the corresponding data points as initial centroids. As an example you can think we can manually select any first 4 values from the synt data.
Returns the initial centroids.

In [ ]:
def initialize_centroids(data, k):
    # Randomly select k data points as initial centroids
    indices = np.random.choice(len(data), k, replace=False)
    centroids = data[indices]
    return centroids

**Function No 2**  `assign_to_clusters`:   This function assigns each data point to the cluster of the nearest centroid.

**Input:**
- data: The input data points.
- centroids: The current centroids.

**Implementation:**

Computes the *Euclidean distances* from each data point to each centroid.
Assigns each data point to the cluster of the nearest centroid.

***Returns*** an array of cluster assignments.

As suggestion, you can plot what returns, would you be able to make it?

In [ ]:
def assign_to_clusters(data, centroids):
    # Compute distances from each data point to each centroid, linalg is a popular way to do it, find out more in the numpy documentation.
    distances = np.linalg.norm(data[:, np.newaxis] - centroids, axis=2)

    # Assign each data point to the cluster of the nearest centroid
    clusters = np.argmin(distances, axis=1)
    return clusters

**Function 3:** - `update_centroids` : This function updates the centroids based on the **mean** of data points in each cluster.

**Input:**
- data: The input data points.
- clusters: The current cluster assignments.
- k: The number of clusters.

**Implementation:**

Iterates over each cluster.
Computes the mean of data points in each cluster to get the new centroid.

**Returns** an array of updated centroids.

In [ ]:
def update_centroids(data, clusters, k):
    # Update centroids based on the mean of data points in each cluster
    centroids = np.array([data[clusters == i].mean(axis=0) for i in range(k)])
    return centroids

**Function 4** `k_means`: The main K-means algorithm function, which iteratively initializes centroids, assigns data points to clusters, and updates centroids until convergence or reaching a maximum number of iterations.

**Input:**

- data: The input data points.
- k: The number of clusters.
- max_iterations: Maximum number of iterations (default is 100).

**Implementation:**
Initializes centroids using initialize_centroids.

Iterates through the main steps of K-means (assigning to clusters and updating centroids) until convergence or reaching the maximum number of iterations.

**Returns** the final cluster assignments and centroids.

In [ ]:
def k_means(data, k, max_iterations=100):
    # Step 1: Initialize centroids
    centroids = initialize_centroids(data, k) #See how I can call a function inside another one.

    for iteration in range(max_iterations):
        # Step 2: Assign data points to clusters
        clusters = assign_to_clusters(data, centroids) # again in here.

        # Step 3: Update centroids
        new_centroids = update_centroids(data, clusters, k)

        # Check for convergence (Again optional but just in case)
        if np.all(centroids == new_centroids):
            break

        centroids = new_centroids #Is this part of the loop?, what am I doing here, can you describe?


    return clusters, centroids


This part of the code generates synthetic data for demonstration purposes, runs the K-means algorithm with k=3, and visualizes the results.

**Implementation:**

- Concatenates three sets of samples from a normal distribution to create
synthetic data.
- Calls the k_means function to obtain cluster assignments and final centroids.
- Plots the data points, colored by cluster, and highlights the final centroids.



In [ ]:
# Generate synthetic data for demonstration
np.random.seed(11)
data = np.concatenate([np.random.normal(loc=0, scale=1, size=(150, 2)),
                       np.random.normal(loc=5, scale=1, size=(150, 2)),
                       np.random.normal(loc=10, scale=1, size=(150, 2))])

# Run K-means algorithm with k=3
k = 3
clusters, final_centroids = k_means(data, k)

# Visualize the results
plt.scatter(data[:, 0], data[:, 1], c=clusters, cmap='viridis', alpha=0.7)
plt.scatter(final_centroids[:, 0], final_centroids[:, 1], c='red', marker='X', label='Centroids')
plt.title('K-means Clustering - Manual implementation')
plt.xlabel('X Axis')
plt.ylabel('Y Axis')
plt.legend()
plt.show()


Please now run the code, and experiment with different parameters, and observe how the algorithm converges to cluster the synthetic data.

You can start exploring the impact of changing the number of clusters (**`k`**) and the number of iterations (**`max_iterations`**). to see how the plot, clusters and centroids are updated. Feel free to add more code cell above so you can plot and expore all the changes you want.

# K-Means implementation using the sklearn library
---
Even thougth the code implemented above isn't complicated, and works well for synth data (small) and educational propuses, you don't want to write this amount of code when you want to process big and multidimensional datasets.

Thankfully 🤟 we have the **sklearn** library, now let's see how easy is to run K-Means in just a few steps.


In [ ]:
# Generate synthetic data
np.random.seed(42)
x = np.random.rand(300) * 10
y = np.random.rand(300) * 10
data = pd.DataFrame({'X': x, 'Y': y})

# Create GeoDataFrame from the synthetic data. Now in this case we will GeoPandas., notice we have not defined any CRS, but that's ok.
geometry = gpd.points_from_xy(data['X'], data['Y'])
gdf = gpd.GeoDataFrame(data, geometry=geometry)

# Visualize the synthetic data
gdf.plot(marker='p', color='blue', figsize=(8, 8))
plt.title('Synthetic Spatial Data')
plt.xlabel('X-axis')
plt.ylabel('Y-axis')
plt.show()

Please address the following questions after having a look at the code from the previous cell.

1. What is the purpose of **`np.random.seed(42)`**?

2. What is the role of **`gpd.points_from_xy`**?

3. What does **`gdf.plot(marker='o', color='blue')`** do?



---

Write your responses here ( `edit this cell`):

---



In [ ]:
# Implement K-means clustering
kmeans = KMeans(n_clusters=3, random_state=42)
gdf['kmeans_cluster'] = kmeans.fit_predict(gdf[['X', 'Y']])

# Visualize K-means clustering result
gdf.plot(column='kmeans_cluster', categorical=True, legend=True, figsize=(8, 8), cmap='Set1')

# Plot the centroids as a white X
centroids = kmeans.cluster_centers_
plt.scatter(
    centroids[:, 0],
    centroids[:, 1],
    marker="x",
    s=169,
    linewidths=3,
    color="b",
    zorder=10,
)
plt.title('K-means Clustering Result')
plt.xlabel('X-axis')
plt.ylabel('Y-axis')
plt.show()


After exploring how easy is to implement **K-Means using this popular ML library**. Could you explain the code above and answer the following questions:

1. What is the purpose of **`KMeans(n_clusters=3, random_state=42)`**?
2. What does **`kmeans.fit_predict(gdf[['X', 'Y']])`** do?
3. What does **`gdf.plot(column='kmeans_cluster', categorical=True, legend=True, cmap='Set1')`** do?




---


Write your responses here:


---



# DBSCAN manual implementation

Now it's the turn for the DBSCAN full implementation, read carefully the code and the description of each function to dive into how the data is being processed, we will also create a set of synthetic data and a set of multiple functions to help you understand what is happeing behind the cortains when you apply this clustering method.


**Function 1:** `find_neighbors`:  This function finds neighbors of a given data point within a specified distance (epsilon).

**Input:**
- data: The input data points.
- point_index: Index of the data point for which neighbors are to be found.
- epsilon: Maximum distance to consider for neighbors.

**Implementation:**

Computes distances from the given point to all other data points.
Selects indices of data points within the specified distance (epsilon).

**Returns** the indices of neighboring points.



In [ ]:
def find_neighbors(data, point_index, epsilon):
    # Find indices of data points within epsilon distance from the given point
    distances = np.linalg.norm(data - data[point_index], axis=1)
    neighbors = np.where(distances <= epsilon)[0]
    return neighbors


**Function 2:** `expand_cluster`: This function is sligthly more complex that the one we have integrated before the function expands a cluster by assigning points to the current cluster and recursively exploring their neighbors.

**Input:**
- data: The input data points.
- point_index: Index of the current data point.
- neighbors: Indices of neighbors of the current data point.
- cluster_id: Identifier for the current cluster.
- epsilon: Maximum distance to consider for neighbors.
- min_samples: Minimum number of neighbors to consider a point a core point.
- clusters: Array indicating the cluster assignment of each data point.
- visited: Array indicating whether a data point has been visited.

**Implementation:**

Initialy we need to assigns the current point to the current cluster., then we need to expands the cluster by recursively exploring neighbors and their neighbors., finally we must check if each neighbor has enough neighbors to be considered a core point., if not we dismiss that iteration., As this method include noise, so the last step is to assigns each unassigned neighbor to the current cluster (most likely to become noise)


In [ ]:
def expand_cluster(data, point_index, neighbors, cluster_id, epsilon, min_samples, clusters, visited):
    # Assign the point to the current cluster
    clusters[point_index] = cluster_id

    # Expand the cluster by iterating over neighbors
    for neighbor_index in neighbors:
        if not visited[neighbor_index]:
            visited[neighbor_index] = True
            new_neighbors = find_neighbors(data, neighbor_index, epsilon)

            # Check if the neighbor has enough neighbors to be a core point
            if len(new_neighbors) >= min_samples:
                neighbors = np.union1d(neighbors, new_neighbors)

        # Assign the neighbor to the current cluster if not assigned to any cluster
        if clusters[neighbor_index] == -1:
            clusters[neighbor_index] = cluster_id


**Function 3:** `dbscan`: We have come to the place where we need to call the created functions, here we are defining the main DBSCAN algorithm function, which iterates through data points, expands clusters, and assigns points to clusters.

**Input:**
- data: The input data points.
- epsilon: Maximum distance to consider for neighbors.
- min_samples: Minimum number of neighbors to consider a point a core point.

**Implementation:**

Initializes variables for cluster assignments and visited points. Notice how this methods wont require a defined number of clusters. Even though both methods are classifed as unsupervised clustering methods, as you don't need any traning dataset to define the size and shape of clusters., then the funcion will iterates through each data point and expands clusters using `expand_cluster`.

**Returns** the array of cluster assignments. Think about the categories or label from each cluster.


In [ ]:
def dbscan(data, epsilon, min_samples):
    # Initialize variables
    num_points = len(data)
    clusters = np.full(num_points, -1)  # -1 represents unassigned points
    visited = np.full(num_points, False)

    # Initialize cluster ID
    cluster_id = 0

    for point_index in range(num_points):
        if not visited[point_index]:
            visited[point_index] = True
            neighbors = find_neighbors(data, point_index, epsilon)

            # Check if the point is a core point
            if len(neighbors) >= min_samples:
                expand_cluster(data, point_index, neighbors, cluster_id, epsilon, min_samples, clusters, visited)
                cluster_id += 1

    return clusters

**Putting all together and plot**

Finally the code bellow will generates synthetic data for demonstration purposes, we will use the same methods we run previously, then runs the DBSCAN algorithm, and visualizes the results using the `matplotlib`. Please feel free to add new code cell and plot the intermediate results, or data to get a sense of how the data is being manipulated.


In [ ]:
# Generate synthetic data for demonstration.

np.random.seed(42)
data = np.concatenate([np.random.normal(loc=0, scale=1, size=(100, 2)),
                       np.random.normal(loc=5, scale=1, size=(100, 2)),
                       np.random.normal(loc=10, scale=1, size=(100, 2))])

# Run DBSCAN algorithm with epsilon=1 and min_samples=5
epsilon = 1
min_samples = 5
clusters = dbscan(data, epsilon, min_samples)

# Visualize the results
plt.scatter(data[:, 0], data[:, 1], c=clusters, cmap='viridis', alpha=0.7)
plt.title('DBSCAN Clustering - Manual Implementation')
plt.xlabel('X Axis')
plt.ylabel('Y Axis')
plt.legend()
plt.show()

# DBSCAN implementation using the sklearn library
---

Same as **K-Means**, you don't want to write large portions of code particually when it comes to process big and multidimensional datasets. Let's see how easy is to run **DBSCAN** in just a few steps.

In [ ]:
# Generate synthetic spatial data for practice
np.random.seed(42)
x = np.random.rand(300) * 10
y = np.random.rand(300) * 10
data = pd.DataFrame({'X': x, 'Y': y})

# Create GeoDataFrame from the synthetic data
geometry = gpd.points_from_xy(data['X'], data['Y'])
gdf = gpd.GeoDataFrame(data, geometry=geometry)

# Visualize the synthetic data
gdf.plot(marker='p', color='blue', figsize=(8, 8))
plt.title('Synthetic Spatial Data')
plt.xlabel('X-axis')
plt.ylabel('Y-axis')
plt.show()

In [ ]:
# Implement DBSCAN clustering
dbscan = DBSCAN(eps=0.8, min_samples=5)
gdf['dbscan_cluster'] = dbscan.fit_predict(gdf[['X', 'Y']])

# Visualize DBSCAN clustering result
gdf.plot(column='dbscan_cluster', categorical=True, legend=True, figsize=(8, 8), cmap='Set1')
plt.title('DBSCAN Clustering Result - With sklearn')
plt.xlabel('X-axis')
plt.ylabel('Y-axis')
plt.show()

As you have noticed the key in **DBSCAN** is the definition of **eps(epsilon)** and the **min_samples**( minimum points).

In a relative recent publicacion (1), authors suggests to use a larger **min_samples** for *large and noisy data sets*, and to adjust **eps** (epsilon) depending on whether you get too large clusters (decrease epsilon) or too much noise (increase epsilon).

It is important you understand that ***clustering requires iterations***, so please take time to play with those values so you can see the difference on the outcomes.

```
(1) Schubert, E., Sander, J., Ester, M., Kriegel, H. P., & Xu, X. (2017).
DBSCAN Revisited, Revisited: Why and How You Should (Still) Use DBSCAN.
ACM Transactions on Database Systems (TODS), 42(3), 19.
```


---




Please address the following questions:

1. What is the purpose of **`DBSCAN(eps=1.5, min_samples=8)`**?
2. Have you tried to update/change the **nmin_sample** size?
3. What does **`dbscan.fit_predict(gdf[['X', 'Y']])`** do?
4. What does **`gdf.plot(column='dbscan_cluster', categorical=True, legend=True, cmap='Set1')`** do?




---
Write your responses here:


---




# Challenges - Let's get your coding skills tested.

I know that so far you have been running the code cells and getting most of the results. Now, it's time to do some thinking, try writing your own code, and correct some common mistakes.

## Fix the code
Please read carefully the following code cells and try to address the issues included in the code.

In [ ]:
np.random.seed(42)
x = np.random.rand('150') * 10
y = np.random.rand('300') * 10
data = pd.DataFrame({'X': x, 'Y': y})

gdf.plot(marker='s', color='r', figsize=(8, 8))
plt.title('Synthetic Spatial Data'):
  plt.xlabel('X-axis')
  plt.ylabel('Y-axis')
  plt.show()

In [ ]:
# What would happen if you update the n_clusters from 3 to 4.
kmeans = KMeans(n_clusters=4, random_state=42)
gdf['kmeans_cluster'] = kmeans.fit_predict(gdf[['X', 'Y']])

# Check this link https://matplotlib.org/stable/users/explain/colors/colormaps.html
# and try to make the plot with another cmap, instead of the viridis

gdf.plot(column='dbscan_cluster', categorical=True, legend=True, figsize=(8, 8), cmap='viridis')
plt.title('DBSCAN Clustering Result'):
  plt.xlabel('X-axis')
  plt.label('Y-axis')
  plt.show()

# Applying Spatial Clustering to real data

Now, let's try to implement both clustering algoritms to analyze real data, related to reported crime in the UK. This data can be found in the following link https://data.police.uk/data/, you can select a range of time and then the force that you would like to get the data. For this example we will work with the reported crime from **Jan 2023 to Sep 2023** collected by the Metropolitan Police Service.



Go to Moodle and download the **crime_data_2023_london** file. Unzip the file, then upload it to your Google Drive so you can access it from this notebook. At this stage, you should already know how to find the path to any uploaded file and use it in your notebook.

## K-Means & DBSCAN - Implementation

## Reading the Data

Initally we need to read the data., make sure you upload the data that was provided in Moodle in your Google drive so you can update the path below.

In [ ]:
# Load crime data for London
crime_data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/CrimeLondon/Crime/london_crime_2023.csv", dtype={"Location": str, "LSOA code": str, "LSOA name": str, "Crime type": str })
crime_data.head()

## Pre-Processing

We always want to check the size of the data we load in memory, as `crime_data`, you see you have loaded more than 96.000 rows and 12 columns.

In [ ]:
crime_data.shape

There are a lot of columns in this dataset:

In [ ]:
crime_data.columns

Let's keep only specific columns that we care about:

In [ ]:
keep_cols = [
    "Month",
    "Latitude",
    "Longitude",
    "Location",
    "LSOA code",
    "LSOA name",
    "Crime type",
]
crime_data = crime_data[keep_cols]
crime_data.head()

## Removing Null values

Before processing this data any further, we need to make sure we don't have any **Null** rows in your **Latitude** and **Longitude** columns. A very easy way to do it, is by using the isnull() function and using **mean** to calculate the fraction of records in **crime_data** where either the latitude or longitude is missing. This combines the two Boolean Series element-wise, returning True if either latitude or longitude is missing in that row.



In [ ]:
(crime_data["Latitude"].isnull() | crime_data["Longitude"].isnull()).mean()

When applied to a Boolean Series, True is treated as 1 and False as 0.
So taking the mean gives you the proportion (fraction) of rows that have a missing latitude or longitude.

Removing the Null rows

In [ ]:
crime_data = crime_data[(crime_data["Longitude"].notnull() & crime_data["Latitude"].notnull())]

Try again the mean is Zero

In [ ]:
(crime_data["Latitude"].isnull() | crime_data["Longitude"].isnull()).mean()

In [ ]:
crime_data.head()

Ok, now the data is clean, and we make sure there are no gaps in the attributes we need to geocode this table and transform it into a GeoPandas DataFrame.

## Construct the GeoDataFrame

Now let's construct a GeoDataFrame from this data.

In [ ]:
geometry = gpd.points_from_xy(crime_data.Longitude, crime_data.Latitude)
gdf_crime = gpd.GeoDataFrame(crime_data, geometry=geometry, crs="EPSG:4326")
gdf_crime.head()

you see you have now a new column called **Geometry**, and the table is now spatial, including a defintion of of coordinate system. **EPSG:4326** which is the standard code for the **WGS84 coordinate system**, which is used globally for Latitude and Longitude coordinates (like those from GPS). Now is ready to make a map.

This is a relative large dataset, if we use the tradional `.plot()` or `explore()` we migth face a huge delay. That is why we installed **lonboard** to represent large datasets like this one. A very simple way to make this map is using `viz(name_of_geodataframe)`

In [ ]:
viz(gdf_crime)

You may have noticed that even though the dataset is supposed to be about car accidents in **London**, there are points scattered across the UK. Therefore, we still need to perform additional data cleaning.

## More cleaning...we need also a Spatial filter

We can check the boundaries of this dataset, with the function `total_bounds`.

In [ ]:
gdf_crime.total_bounds

This is clearly not the spatial boundaries of london.

By using the tool https://norbertrenner.de/osm/bbox.html to get the precise boundary box you need to spatially filter your analysis and remove any outlier rows that are not located in the area of interest.

In [ ]:
ld_bbox = [-0.591,51.285,0.381,51.678] # London B. Box

Slicing the dataset based on the new boundaries.

In [ ]:
gdf_crime = gdf_crime[gdf_crime.intersects(shapely.box(*ld_bbox))]

By using `.shape` we can check the new amount of rows in the spatial dataset

In [ ]:
gdf_crime.shape

Now we can use again the [lonboard](https://developmentseed.org/lonboard/latest/) library to valide if our spatial filter worked.


There are two methods in this very recent and under development library `viz(gdf)` which as we could see before is a quick method to plot your big datasets.

The second one where we have more control of how the layer would look like `ScatterplotLayer`, thus we can create a `layer` then use other methods in the library to customize the visualization of the layer, you could also load several layers, including lines and polygons.

In [ ]:
viz(gdf_crime)

Now lets try the second method to plot a map using lonboard

In [ ]:
layer = ScatterplotLayer.from_geopandas(gdf_crime)
map = Map(layers=[layer], _height=500)
map

**Did you get an error here? What do you think is missing?**

**Do we need to import something important that we haven't imported yet?,**

Have a look at the library documentation website https://developmentseed.org/lonboard/latest/examples/internet-speeds/#dependencies and see if you can identify the **import** we are missing.

In [ ]:
#Add the Fixed code in here.

## Customizing the Layer

Now we can customize the previous layer by plotting by `Crime type` in an initial attemtp to unhide any spatial pattern. Let's initially check how many catgeories we have in this `Crime type` column.

In [ ]:
crime_type = gdf_crime['Crime type'].value_counts()
crime_type

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns
gdf_crime.groupby('Crime type').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))

We can see that "Violence and sexual offences" is the **crime type** with the most reports. Now, let's check for any spatial patterns by plotting these categories on our previously loaded map.

To do this, we need to assign a color to each crime type, which we can accomplish by creating an array of all the unique categories.



In [ ]:
categories = gdf_crime['Crime type'].unique()
colors = sns.color_palette("bright", len(categories))
colors

Now using the same principle, we can use Numpy to create a matrix that group every `Crime type` with its correspondant `color code. ` This code will use the coding `RGBA(Red, Green, Blue,Alpha)` the last value corresponde to transparency.

In [ ]:
# Get unique categories
categories = gdf_crime['Crime type'].unique()

# colour seaborn's "tab10" color palette
colors = sns.color_palette("bright", len(categories))

# Create a dictionary to map categories to colors
color_dict = dict(zip(categories, colors))

color_array = np.array([tuple(np.append(np.multiply(color_dict.get(x, (0, 0, 0)), 255).astype(int), 255)) for x in gdf_crime['Crime type']], dtype=np.uint8)
color_array

With this new array containing the color codes for each crime type category, you can now use the layer's properties—such as `radius_scale`, `opacity`, and `get_fill_color`—to customize the map.

**Run the following code, and return to the map you previously plotted (the one where you have corrected the code) to view the updated, color-coded layer. You probably need to zoom it to see the points associated to the Crime Type Categories**

In [ ]:
layer.radius_scale = 40
layer.opacity = 0.05
layer.get_fill_color = color_array

## Density Surface - HeatMap

As you may have noticed, even though a choropleth map provides some insight, it can still be difficult to detect clear spatial patterns.

Let's try using the heatmap option from Geopandas to better visualize the density and distribution of our data.

In [ ]:
import folium
from folium import plugins

map = folium.Map(location=[51.518591, -0.108447], tiles="Cartodb dark_matter", zoom_start=9)

heat_data = [[point.xy[1][0], point.xy[0][0]] for point in gdf_crime.geometry]
plugins.HeatMap(heat_data).add_to(map)
map

**Do you think the previous HeatMap is some how useful?**

**Can you unveail any hidden spatial pattern?**

I guess your reply is no.

**HeatMaps** are useful as initial step to explore any dense areas based on the location of your dataset, but still does not provide any relevant insigth, as we saw in the previous lecture.

## Reducing the Subjectivity of the Map - Spatial Clustering

Now we can implement **K-Means** and **DBSCAN** by using the sklearn library, and then plot the results using a popular library called Plotly Express - https://plotly.com/python/plotly-express/.

In [ ]:
# Let's start with KMeans.
kmeans = KMeans(n_clusters=5, random_state=42)
gdf_crime['kmeans_cluster'] = kmeans.fit_predict(gdf_crime[['Longitude', 'Latitude']])


In [ ]:
import plotly.express as px

fig_kmeans = px.scatter_mapbox(
    gdf_crime,
    lat="Latitude",
    lon="Longitude",
    color="kmeans_cluster",
    mapbox_style="carto-positron",
    zoom=9,  # Adjust zoom level as needed
    title="K-means Clustering",
    height=700,    # Set initial height in pixels (width is responsive)
    opacity=0.5,

)
fig_kmeans.show()

Now, to make the spatial clustering with DBSCAN, there is a fundamental step to make it correctly. Do you recall what is the crs defined in `gdf_crime`?

In [ ]:
print(gdf_crime.crs)

You should have seen **EPSG:4326**, which is a geographic coordinate system. That means you have your data using Latitude and Longitude.

Can you calculate distances using Latitude and Longitude, like the **eps (epsilon)** needed in the DBSCAN implementation?

As I really hope your reply was **NO**. Then we need to reproject our geopandas dataframe and transform it to use a local and projectec reference system. The EPSG code for the UK is **EPSG:27700**, which represents the **British National Grid (BNG)**. This coordinate system is used for onshore Great Britain and the Isle of Man, and it is based on the OSGB36 geodetic datum. https://epsg.io/

In [ ]:
gdf_crime_projected = gdf_crime.to_crs(epsg=27700)
gdf_crime_projected.head()

 **DBSCAN** use the X and Y coordinates of your dataset, but we don't have those columns, so we need to use the `geometry` column to extract these X/Y coordinates for DBSCAN:

In [ ]:
gdf_crime_projected['geometry_x'] = gdf_crime_projected.geometry.x
gdf_crime_projected['geometry_y'] = gdf_crime_projected.geometry.y
gdf_crime_projected.head()

Now we can define the most relevant parameters of the DBSCAN implementation.

*   meters_eps = 200 (200 meters)
*   min_samples_val = 40 (Min of 40 points)

In [ ]:
meters_eps = 200
min_samples_val = 40

dbscan = DBSCAN(eps=meters_eps, min_samples=min_samples_val)

gdf_crime_projected['dbscan_cluster'] = dbscan.fit_predict(
    gdf_crime_projected[['geometry_x', 'geometry_y']])

gdf_crime_projected.head()


Let's map the final results, using Lat and Long columns (requiered by plotly express)

In [ ]:
fig_dbscan = px.scatter_mapbox(
    gdf_crime_projected,
    lat="Latitude",
    lon="Longitude",
    color="dbscan_cluster",
    mapbox_style="carto-positron",
    zoom=9,  # Adjust zoom level as needed
    title="DBSCAN Clustering",
    height=700,    # Set initial height in pixels (width is responsive)
    opacity=0.5,

)
fig_dbscan.show()

Now we can start adjusting the **required parameters** to perform multiple evaluations.

**Finally:** Take some time **to adjust** **and test** **several values** for both **KMeans** and **DBSCAN** and observe how the spatial clustering changes.

This way, you are minimising the **subjectivity of your map** by including spatial properties of your data; still, you need to work with meaningful arguments to define the key parameters. That is why —**there is no such thing as removing the subjectivity of maps**-now, you can reduce it by implementing a **machine learning technique** into large datasets to reveal hidden patterns.

Recall this clustering include only the location of the reported crimes. You can eventually also create clusters by using a **categorial attribute** like **type of crime** and see if there is any spatial correlation with areas where those crimes were reported.

# Conclusions

Our exploration into spatial clustering using the **KMeans** and **DBSCAN** algorithms has provided valuable insights into uncovering patterns within crime data in London. The visualizations created with `Plotly` and `GeoPandas` have allowed us to observe how these algorithms group spatial data points based on their geographical coordinates.

This brief introduction to spatial clustering offers just a glimpse into the world of data-driven spatial analysis. The use of **KMeans**, a partitioning and unsupervised clustering method, and **DBSCAN**, a density-based clustering approach, also unsupervised method,  has display distinct patterns within the crime dataset. We've witnessed the power of these algorithms in identifying clusters and anomalies, offering a foundation for understanding spatial distributions. It was also relevant to see how useful using external and powerful Machine Leaning libraries like `Sklearn` can be in python, and how much  code would eventually save us.

**It is important to note that spatial clustering extends far beyond the methods explored here. Statistical and spatial clustering techniques provide a diverse toolkit for researchers and analysts, enabling a deeper understanding of underlying patterns.**

**If you want to continue your learning and practice into spatial data analysis, in the advance module (GG4257)** I will cover more libaries, how to install locally the python stack, and we will cover more methods that go beyond the geographic proximity, incorporating statistical measures and advanced clustering algorithms.

As final step, now is your turn to try some of these code lines with the **Exercise notebook available in Moodle.**

**Aditional Note:** The library we used called lonboard https://developmentseed.org/lonboard/latest/ and you have installed at the beinging of this notebook allow us to load much larger datasets, around 2 to 4 millons of records, something that would be nearly impossible to do it with the tradtional GIS tools and your computer as we will overpass the computational capacity in a matter of seconds.

Here in Google Colab you also have some **RAM and Disk limits**, so you probably will need to be aware of how you manipulate load big datasets in memory.

In general terms we can do plenty of analysis here, but unless we are willing to set up a credit card and pay for more cloud resources, the best way to have **full control and use more RAM or Disk resources** is installing the so-called python stack in our machines, so we can use our local resources without paying anything extra, the limit will be defined by the RAM and Disk limits on your machine.